In this notebook, we will take our previous Elo system for tennis players and add playing surface as a parameter. There are a few ways in which surface has been taken into account.

1. (Surface-only) treat each surface as a different sport altogether, so that each player has three ratings that don't interact with one another.
2. (Weighted average) take the surface-specific ratings in item 1 above and the all-surfaces ratings developed in our previous post, then take a weighted average of them, minimizing the log-loss error.
3. (Surface-dependent K-factor) According to the surface being played on, update each player's surface-specific rating according to a different K-factor and take the win probability from the corresponding surface-specific ratings.

The first and second are implemented by Jeff Sackmann, the tennis data god, where the weighted average is the actual average. The third is the idea introduced in this post, which seems fairly natural to me and perhaps a little less ad-hoc than taking the average between surface-only and surface-agnostic ratings. So let's explain how the surface-dependent K-factor (SDKF) model works.

## SDKF model

Define $$\sigma(x) = \exp(x) / (\exp(x) + 1),$$ the logistic function. If player one (p1) has rating $x$ and player two (p2) has rating $y$, the probability that p1 wins is given by $\sigma(x-y)$. Suppose $w=1$ if p1 wins and $w=0$ if p1 loses. After the match, the ratings are updated with the rule $$x \mapsto x + (-1)^{w+1} K(n_1)\sigma((-1)^w(x-y)),\quad y \mapsto y+(-1)^w K(n_2)\sigma((-1)^w (x-y)),$$ where $K$ is a function of the number of matches played by p1 ($n_1$) and the number of matches played by p2 ($n_2$). The function $K$ is of the form $$K(n) = \frac{a}{(b + n)^c}.$$

To define surface-specific ratings, we can do the following. Let $A$ be a $3\times 3$ matrix. We map surfaces to indices: index 1 refers to clay, 2 to grass, 3 to hard. Now let $\vec{x},\vec{y}\in \mathbb{R}^3$ be the ratings of p1 and p2, respectively. Specifically, $$\vec{x} = (x_1,x_2,x_3)$$ and $x_1$ is the p1 clay rating, $x_2$ is the p1 grass rating, and so on. Define $\sigma(\vec{x}) = (\sigma(x_1),\sigma(x_2),\sigma(x_3))$. If $a_{ij}$ is the $(i,j)$ entry of $A$, then we make the following change to the update rule: $$\vec{x} \mapsto \vec{x} + (-1)^{w+1}K(n_1)A\sigma((-1)^w(\vec{x}-\vec{y})), \quad \vec{y} \mapsto \vec{y} + (-1)^w K(n_2)A\sigma((-1)^w(\vec{x}-\vec{y})).$$

The matrix $A$ consists of the speed with which to update each of the three ratings, given the surface being played on. For example, if the match is being played on grass, we intuit that the result shouldn't have a large effect on the players' clay rating, but it should have a large effect on the players' grass rating. On the other hand, if the match is being played on hard, we might think that it should have an equal effect on the players' grass and clay ratings.

Finally, let's determine the win probability and the interpretation of the matrix $A$. If $$\vec{s}=\begin{cases} \vec{e}_1 &\quad \text{ if clay} \\ \vec{e}_2 &\quad \text{ if grass} \\ \vec{e}_3 &\quad \text{ if hard} \end{cases}$$ is the vector denoting surface being played on, then the win probability of p1 is $$\sigma(\vec{x}-\vec{y})\cdot \vec{s}.$$ This indicates that **$a_{ij}$ is the update speed for the players' surface $i$ rating if the playing surface is $j$**.

### Special cases
It is instructive to examine special cases of $A$.
1. If $A$ is the identity matrix, then no surface affects any other surface, and all the update coefficients are equal. So this would be equivalent to treating each surface as a different sport altogether (Surface-only ratings).
2. If $A$ is the all-ones matrix, then all surfaces are treated equally. This results in surface-agnostic ratings, which is the classical setting.

Based on these two extremes, we expect an effective $A$ to have heavy diagonal entries but nonzero off-diagonal entries, all positive. For our $K$, we take $a,b,c$ to be $$(a,b,c) = (0.47891635, 4.0213623 , 0.25232273)$$ based on training data from 1980 to 2014, from the previous post. Then we initialize the entries of $A$ to be uniform random numbers between 0 and 1.5.

In [2]:
import numpy as np
import pandas as pd

np.random.seed(5192020) # today's date!

def sigmoid(z):
    '''The sigmoid function.'''
    return np.where(z >= 0, 
                    1 / (1 + np.exp(-z)), 
                    np.exp(z) / (1 + np.exp(z)))

class Elo_sdkf:
    
    def __init__(self, start_year, end_year, num_models, 
                 k_param = np.array([0.47891635, 4.0213623 , 0.25232273]),
                 lower = np.zeros(9), upper = np.ones(9) * 1.5):
        
        self.start_year = start_year
        self.end_year = end_year
        self.num_models = num_models
        
        self.k_params = np.multiply(np.ones((num_models, 3)), k_param)
        self.a_params = np.multiply(np.random.random((num_models, 9)), upper - lower) + lower
        
        self.data = []
        for i in range(start_year, end_year + 1):
            self.data.append(pd.read_csv('./atp/atp_matches_' + str(i) + '.csv'))
        
        # collect all the player names
        self.players = {player for player in self.data[0]['winner_name']}
        self.players = self.players.union({player for player in self.data[0]['loser_name']})
        for i in range(1, end_year - start_year + 1):
            self.players = self.players.union({player for player in self.data[i]['winner_name']})
            self.players = self.players.union({player for player in self.data[i]['loser_name']})            
        
        # ratings are of the form (n, r)
        # where n is the number of matches the player has played
        # and r is their rating.
        self.ratings = {player: (0, np.ones((num_models,3))) for player in self.players}
        self.select_ratings = {player: (0,1.0) for player in self.players}
        
        col_names = ['a'+str(i) for i in range(1,10)]
        self.params_data = pd.DataFrame({a : self.a_params[:,i] for (a,i) in zip(col_names, range(9))})
        self.params_data['k1'] = self.k_params[:,0]
        self.params_data['k2'] = self.k_params[:,1]
        self.params_data['k3'] = self.k_params[:,2]
        self.params_data['ll'] = np.zeros(num_models)
        self.params_data['wp'] = np.zeros(num_models)
        self.params_data['bs'] = np.zeros(num_models)
        
    def get_params(self):
        return self.k_params.copy(), self.a_params.copy()
    
    def set_params(self, k_ps, a_ps):
        self.k_params, self.a_params = k_ps, a_ps
        
    def get_params_data(self):
        return self.params_data.copy()
    
    def k(self, n, ps):
        '''returns the vector K-factor, which dictates how sensitive ratings are
        to an individual match and depends on the number of matches played.'''
        return np.multiply(ps[:,0], 
                           np.power(ps[:,1] + n, -ps[:,2])
                          )
    
    
    def update_one(self, x, y, n1, n2, k_params, a_params, s):
        '''this function updates one match. 'x','y' are the ratings of
        the winner and loser respectively, 'n1','n2' are the number of matches
        that the winner and loser have played respectively. Returns the
        prior probability that the winner wins, and the values to update 
        the ratings by. '''
        z = np.multiply(np.dot(a_params.reshape((len(a_params),3,3)), s), sigmoid(y-x))
        z1 = z[:,0]
        z2 = z[:,1]
        z3 = z[:,2]
        u1 = np.multiply(self.k(n1, k_params), z1)
        u2 = np.multiply(self.k(n1, k_params), z2)
        u3 = np.multiply(self.k(n1, k_params), z3)
        v1 = -np.multiply(self.k(n2, k_params), z1)
        v2 = -np.multiply(self.k(n2, k_params), z2)
        v3 = -np.multiply(self.k(n2, k_params), z3)
        
        u = np.transpose(np.array([u1,u2,u3]))
        v = np.transpose(np.array([v1,v2,v3]))
        prob = np.dot(sigmoid(x-y), s)
        return(prob, u, v)
    
    def update_all_ratings(self, year):
        '''update all the ratings at once. '''
        # first reset the ratings.
        self.ratings = {player: (0, np.ones((self.num_models,3))) 
                            for player in self.players}
        
        ll = np.zeros(self.num_models)
        wp = np.zeros(self.num_models)
        bs = np.zeros(self.num_models)
        
        for i in range(len(self.data)):
            for j, row in self.data[i].iterrows():
                
                winner = row['winner_name']
                loser = row['loser_name']
                surface = row['surface']
                if surface == 'Clay':
                    s = np.array([1,0,0])
                elif surface == 'Hard':
                    s = np.array([0,0,1])
                else: # Carpet gets classified as Grass.
                    s = np.array([0,1,0])
                
                # get ratings.
                wnm, wrating = self.ratings[winner]
                lnm, lrating = self.ratings[loser]
                
                # update.
                prob_vec, u1, u2 = self.update_one(wrating, lrating, wnm, lnm, 
                                                   self.k_params, self.a_params, s)
                self.ratings[winner] = wnm + 1, wrating + u1
                self.ratings[loser] = lnm + 1, lrating + u2
                
                # compute log-loss error or win prediction percentage.
                if(i + self.start_year >= year):
                    ll -= np.log(prob_vec)
                    wp += (prob_vec > 0.5).astype(float)
                    bs += np.power(1-prob_vec, 2)
        
        # figure out what to divide cost by.
        num_rows = 0
        for i in range(min(len(self.data), self.end_year + 1 - year)):
            num_rows += len(self.data[year - self.start_year - 1 + i])

        self.params_data['ll'] = ll / num_rows
        self.params_data['wp'] = wp / num_rows
        self.params_data['bs'] = bs / num_rows
    
    def sort_params(self, method = 'll'):
        '''sort the parameters according to cost method.'''
        if method == 'll':
            self.params_data = self.params_data.sort_values(by='ll')
        elif method == 'wp':
            self.params_data = self.params_data.sort_values(by='wp', ascending = False)
        else:
            self.params_data = self.params_data.sort_values(by='bs')
    
    def update_select_ratings(self, year, a_params):
        '''this function updates only the ratings corresponding to the given
        parameters. It takes the average probability.'''
        
        # first reset the ratings.
        n = len(self.k_params)
        self.select_ratings = {player: (0, np.ones((n,3))) for player in self.players}
        
        ll = 0
        wp = 0
        bs = 0
        
        for i in range(len(self.data)):
            for j, row in self.data[i].iterrows():
                
                winner = row['winner_name']
                loser = row['loser_name']
                surface = row['surface']
                if surface == 'Clay':
                    s = np.array([1,0,0])
                elif surface == 'Hard':
                    s = np.array([0,0,1])
                else: # Carpet gets classified as Grass.
                    s = np.array([0,1,0])
                
                
                # get ratings.
                wnm, wrating = self.select_ratings[winner]
                lnm, lrating = self.select_ratings[loser]
                
                # update.
                prob, u1, u2 = self.update_one(wrating, lrating, wnm, lnm, 
                                               self.k_params, a_params, s)
                
                # average the ratings
                #prob = sigmoid(np.mean(wrating) - np.mean(lrating))
                self.select_ratings[winner] = wnm + 1, wrating + u1
                self.select_ratings[loser] = lnm + 1, lrating + u2
                
                # what's the prob?
                prob = np.mean(prob)
                
                # compute log-loss error or win prediction percentage.
                if(i + self.start_year >= year):
                    ll -= np.log(prob)
                    wp += (prob > 0.5).astype(float)
                    bs += (1-prob)**2
        
        # figure out what to divide cost by.
        num_rows = 0
        for i in range(min(len(self.data), self.end_year + 1 - year)):
            num_rows += len(self.data[year - self.start_year - 1 + i])

        return (ll / num_rows, wp / num_rows, bs / num_rows)
        

In [22]:
elo_sdkf = Elo_sdkf(2000, 2013, 10000)
elo_sdkf.update_all_ratings(2010)
elo_sdkf.sort_params()
df = elo_sdkf.get_params_data()
print(df[['ll','wp','bs']].head())

            ll        wp        bs
6582  0.575365  0.677100  0.197369
8772  0.575536  0.678666  0.197356
10    0.575539  0.679160  0.197427
3921  0.575581  0.679325  0.197458
7482  0.575583  0.680478  0.197474


In [23]:
col_names = ['a'+str(i) for i in range(1,10)]
print(df[col_names].head())

            a1        a2        a3        a4        a5        a6        a7  \
6582  1.497282  0.709504  0.223734  0.035879  1.016345  0.891814  0.635505   
8772  1.496927  0.122563  0.659796  0.035299  0.704547  1.405535  0.895188   
10    1.459020  0.305191  0.589823  0.331715  1.321166  0.879316  0.332806   
3921  1.459754  0.449260  0.745738  0.556220  1.398105  0.590583  0.627694   
7482  1.299778  0.108285  0.420631  0.155358  1.096234  0.311420  0.437174   

            a8        a9  
6582  1.089314  1.143836  
8772  0.967678  1.212636  
10    0.439226  1.318882  
3921  0.869944  1.246126  
7482  0.778050  0.888988  


Since there are so many parameters now, uniform distribution gets sparser. Let's tighten up our range by taking the top 50 parameters and setting the uniform distribution to be around their means.

In [28]:
print(np.mean(np.array(df[col_names].iloc[:50]), axis=0))
print(np.var(np.array(df[col_names].iloc[:50]), axis=0))

[1.31354461 0.4072288  0.55741823 0.2564878  0.96902885 0.87580268
 0.5603505  0.9463296  1.18036339]
[0.01845077 0.11172311 0.06128348 0.03510118 0.10038243 0.10348859
 0.04403342 0.09945789 0.03073314]


In [32]:
lower = np.mean(np.array(df[col_names].iloc[:50]), axis=0) - 4*np.var(np.array(df[col_names].iloc[:50]), axis=0)
upper = np.mean(np.array(df[col_names].iloc[:50]), axis=0) + 4*np.var(np.array(df[col_names].iloc[:50]), axis=0)

In [4]:
elo_sdkf = Elo_sdkf(1995, 2013, 10000, lower=lower, upper=upper)
elo_sdkf.update_all_ratings(2010)

In [10]:
elo_sdkf.sort_params()
df = elo_sdkf.get_params_data()
print(df[['ll','wp','bs']].head())
elo_sdkf.sort_params(method = 'wp')
df = elo_sdkf.get_params_data()
print(df[['ll','wp','bs']].head())
elo_sdkf.sort_params(method = 'bs')
df = elo_sdkf.get_params_data()
print(df[['ll','wp','bs']].head())

            ll        wp        bs
7482  0.574028  0.681137  0.196757
6900  0.574036  0.680231  0.196770
9358  0.574049  0.680725  0.196766
8063  0.574052  0.680395  0.196785
2516  0.574077  0.680231  0.196772
            ll        wp        bs
3582  0.574796  0.683196  0.197109
8249  0.574623  0.683114  0.197025
3581  0.575047  0.682867  0.197141
2153  0.574277  0.682784  0.196886
7776  0.575089  0.682784  0.197176
            ll        wp        bs
7482  0.574028  0.681137  0.196757
9358  0.574049  0.680725  0.196766
6900  0.574036  0.680231  0.196770
2516  0.574077  0.680231  0.196772
7337  0.574102  0.680643  0.196772


In [11]:
col_names = ['a'+str(i) for i in range(1,10)]
print(df[col_names].head())

            a1        a2        a3        a4        a5        a6        a7  \
7482  1.367645  0.024858  0.449766  0.145167  1.154393  0.633733  0.486885   
9358  1.384846 -0.032520  0.509930  0.138995  0.979521  0.745712  0.409456   
6900  1.373290  0.049444  0.566975  0.195152  1.354489  0.641642  0.534075   
2516  1.325903  0.057243  0.493182  0.125402  1.233855  0.679932  0.557513   
7337  1.364242 -0.026317  0.572670  0.130175  0.971202  0.712183  0.639580   

            a8        a9  
7482  0.961209  1.203145  
9358  1.114731  1.141554  
6900  1.107081  1.138645  
2516  1.257670  1.237305  
7337  1.091639  1.261886  


In [12]:
a_params = np.array(df[col_names].iloc[:50])
elo_sdkf = Elo_sdkf(1980, 2014, 1)
elo_sdkf.update_select_ratings(2014, a_params = a_params)

(0.5796983569656494, 0.6725543478260869, 0.19906761033237047)

In [3]:
a_params = np.array([[ 1.36764503,  0.02485842,  0.44976554,  0.14516707,  1.15439298,
         0.63373323,  0.48688498,  0.96120873,  1.20314498],
       [ 1.38484558, -0.03252025,  0.5099303 ,  0.13899527,  0.97952081,
         0.74571216,  0.40945619,  1.11473126,  1.1415536 ],
       [ 1.37328958,  0.04944392,  0.56697461,  0.19515227,  1.3544885 ,
         0.64164162,  0.53407531,  1.10708059,  1.1386452 ],
       [ 1.32590279,  0.05724343,  0.49318196,  0.1254024 ,  1.2338545 ,
         0.67993168,  0.5575131 ,  1.25767037,  1.23730465],
       [ 1.36424165, -0.02631671,  0.57266992,  0.13017463,  0.9712018 ,
         0.7121833 ,  0.63958048,  1.09163894,  1.261886  ],
       [ 1.38306185,  0.04292298,  0.54624705,  0.14968436,  1.08815874,
         0.57269256,  0.5518662 ,  1.32165028,  1.17445715],
       [ 1.36272796,  0.09102119,  0.48315057,  0.12127602,  1.09455067,
         0.56173307,  0.4647588 ,  1.0868306 ,  1.06824943],
       [ 1.32440032,  0.05123438,  0.48458254,  0.15387142,  1.19423098,
         0.63072074,  0.58159331,  1.08419783,  1.17114495],
       [ 1.36279233,  0.10038846,  0.49869277,  0.14015169,  1.21739132,
         0.54809504,  0.42363132,  1.20853438,  1.22509448],
       [ 1.35947535,  0.01602767,  0.59807534,  0.11879866,  1.29432361,
         0.94653932,  0.62776815,  1.16448067,  1.25011773],
       [ 1.33381355,  0.009458  ,  0.52384005,  0.18678066,  1.2143887 ,
         0.5685181 ,  0.50279126,  1.12618672,  1.17772794],
       [ 1.36528024, -0.03726645,  0.52592846,  0.18620466,  1.21255428,
         0.81701092,  0.60370563,  0.97941152,  1.13068288],
       [ 1.37461081,  0.2869634 ,  0.62145025,  0.12983767,  1.18071768,
         0.66622007,  0.57665516,  1.06242135,  1.19288961],
       [ 1.37348927,  0.01719831,  0.66833825,  0.11982577,  1.14875282,
         0.67315226,  0.61746049,  1.02582283,  1.12350478],
       [ 1.37584264,  0.00194642,  0.55804143,  0.13802308,  1.18729043,
         0.96366695,  0.66001336,  1.06243222,  1.18090345],
       [ 1.33134337,  0.14340834,  0.57954102,  0.14931637,  1.27978691,
         0.77695968,  0.43905136,  1.13833811,  1.17919219],
       [ 1.37960475,  0.11764724,  0.47460025,  0.210259  ,  1.20611241,
         0.51864284,  0.47975398,  1.09269549,  1.17648103],
       [ 1.38338734,  0.22803109,  0.55602508,  0.22021096,  1.31600668,
         0.7878143 ,  0.53162749,  1.00995285,  1.26168343],
       [ 1.33352444, -0.01183061,  0.58660995,  0.14977109,  1.18416053,
         0.49406069,  0.45639573,  1.14837585,  1.1670333 ],
       [ 1.34407927,  0.05750646,  0.53941155,  0.19741456,  1.31545141,
         0.74280224,  0.49840365,  1.2714551 ,  1.28172874],
       [ 1.33897813,  0.03885394,  0.59477974,  0.21650415,  1.22424441,
         0.67215704,  0.50572237,  1.01510021,  1.23269919],
       [ 1.34883714,  0.08618883,  0.65942322,  0.1606466 ,  1.25142431,
         0.74434815,  0.51071121,  0.91390154,  1.18278559],
       [ 1.36296547, -0.03805659,  0.4591379 ,  0.14187701,  0.94716402,
         0.62061573,  0.64598261,  0.93007469,  1.26911017],
       [ 1.37993399, -0.00509933,  0.41462038,  0.14492886,  0.9152252 ,
         0.65388103,  0.62855184,  1.22620705,  1.26161759],
       [ 1.35323588,  0.11655926,  0.52486588,  0.1647503 ,  1.21364914,
         0.59260467,  0.46075745,  0.76657508,  1.21291309],
       [ 1.32426868,  0.04808225,  0.61170129,  0.1455837 ,  1.27211456,
         0.93980087,  0.52493534,  0.97161989,  1.18708546],
       [ 1.36218554,  0.01682465,  0.54829013,  0.15157499,  0.89316736,
         0.99865632,  0.51389574,  1.12667347,  1.17369898],
       [ 1.35550411,  0.0158655 ,  0.59634832,  0.14539788,  0.99945782,
         0.52199803,  0.6485663 ,  1.13082395,  1.27031017],
       [ 1.35442598,  0.26874924,  0.49539648,  0.15433225,  1.31314076,
         0.70234271,  0.6682979 ,  1.07037556,  1.20889975],
       [ 1.3306228 ,  0.0968272 ,  0.5170479 ,  0.12686389,  0.90333132,
         0.86242017,  0.57053466,  0.91757777,  1.23541071],
       [ 1.34359368,  0.11424921,  0.55483679,  0.16145676,  1.08911612,
         0.92700788,  0.47009987,  0.99371029,  1.11704543],
       [ 1.38561576,  0.05370527,  0.35768654,  0.14094665,  1.26313327,
         0.84237536,  0.47401934,  1.17356061,  1.18741258],
       [ 1.36943557,  0.11687515,  0.65763357,  0.1834964 ,  1.27709867,
         0.5316282 ,  0.48819748,  1.10936277,  1.14695513],
       [ 1.34925237,  0.03765416,  0.52944704,  0.12038118,  0.97354592,
         1.00013794,  0.41409654,  1.08648126,  1.22620182],
       [ 1.3325548 ,  0.18900979,  0.44178343,  0.13235153,  0.92096794,
         0.68136218,  0.54475417,  1.08764254,  1.13963685],
       [ 1.38087313,  0.10812382,  0.53190006,  0.12618556,  1.0827559 ,
         0.91986285,  0.72971054,  1.10021161,  1.26116562],
       [ 1.26695141,  0.08547846,  0.50202461,  0.12107583,  1.05190542,
         0.79281625,  0.51824847,  1.02744405,  1.14848515],
       [ 1.36796634,  0.07444662,  0.43065296,  0.19382646,  1.00830384,
         0.74890571,  0.60946505,  0.95295769,  1.23086416],
       [ 1.33889022,  0.15658821,  0.55100683,  0.14721363,  1.06362979,
         0.80944937,  0.4461455 ,  0.87007951,  1.0925754 ],
       [ 1.38349344, -0.01803279,  0.5659115 ,  0.22696616,  1.22828702,
         0.71346806,  0.56550264,  0.80842663,  1.1099554 ],
       [ 1.35424571,  0.01167939,  0.47759777,  0.17271134,  1.33812542,
         0.71372018,  0.73304534,  1.26424741,  1.24353436],
       [ 1.34853549,  0.18457588,  0.43339074,  0.15441289,  0.97948615,
         0.70324615,  0.45941949,  1.20936292,  1.25364145],
       [ 1.33647504,  0.04390857,  0.63434235,  0.18552159,  1.1748817 ,
         0.82625441,  0.41951784,  1.08050471,  1.17264111],
       [ 1.36238702,  0.18002718,  0.44036777,  0.13829458,  1.29106342,
         1.00688228,  0.43957308,  1.17770924,  1.13337054],
       [ 1.38420148,  0.12727853,  0.55309066,  0.17395384,  1.07267802,
         0.95834978,  0.64929306,  1.14443214,  1.15898731],
       [ 1.38137145,  0.01181177,  0.45663072,  0.14383553,  0.87085719,
         1.09536434,  0.49779319,  1.00341078,  1.16412449],
       [ 1.3833151 ,  0.14218627,  0.50506524,  0.17818227,  1.27481556,
         0.94717733,  0.46237453,  0.78148185,  1.27360892],
       [ 1.32927588,  0.13787546,  0.58201061,  0.17637419,  1.2229518 ,
         0.69120687,  0.46202001,  0.81854451,  1.11747007],
       [ 1.35908442,  0.37301553,  0.44537068,  0.12203616,  1.15916478,
         0.71788773,  0.53970979,  0.87917961,  1.10833224],
       [ 1.3860734 ,  0.10684902,  0.61805291,  0.15765615,  1.32209809,
         0.69171828,  0.69604147,  1.19158216,  1.11730913]])

## Testing on 2015-2019 data
Recall the 2015-2019 comparison between the randomly initialized Elo model and the FiveThirtyEight Elo model:
```
  optimized_for        ll        wp        bs
0            ll  0.607112  0.658860  0.209942
1            wp  0.623405  0.657555  0.214121
2            bs  0.607411  0.658723  0.209912
3           538  0.611903  0.661607  0.210791
```

Let's try our SDKF model.

In [4]:
elo_sdkf = Elo_sdkf(1980, 2019, 1)
elo_sdkf.update_select_ratings(2015, a_params = a_params)

(0.6031482224173272, 0.6630494505494505, 0.20829600607130533)